In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import gseapy as gp
import seaborn as sns
import mygene
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
from matplotlib.pyplot import MouseButton
import random

In [5]:
#create networkx graph out of the hippi PPI
hippie = pd.read_csv('hippie.txt', sep='\t', header=None, usecols=[0,1,2,3]) #hippie
hippie.columns = ('id1', 'gene1', 'id2', 'gene2')
hippie_nx = nx.Graph()
hippie_nx.add_edges_from(list(zip(hippie.gene1,hippie.gene2)))

In [6]:
#get gene_info for all genes in hippie
hippie_all_genes = list(set(hippie.gene1).union(set(hippie.gene2)))
mg = mygene.MyGeneInfo()
hippie_gene_info = mg.getgenes(hippie_all_genes, fields = 'symbol')

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

In [7]:
#create a dictionary with entrez ID to symbol conversation for all genes in hippie
hippie_entrez2symbol = dict()
for d in hippie_gene_info:
    try:
        hippie_entrez2symbol[int(d['_id'])] = d['symbol']
    except:
        pass
hippie_symbols = list(hippie_entrez2symbol.values())
print(hippie_symbols)

['A1BG', 'A2M', 'MIX23', 'NAT1', 'NAT2', 'SERPINA3', 'AADAC', 'AAMP', 'AANAT', 'AARS1', 'GAGE2A', 'ABAT', 'ABCA1', 'ABCA2', 'ABCA3', 'ABCB7', 'ABCF1', 'KCNH8', 'ABL1', 'AOC1', 'ABL2', 'ABO', 'ABR', 'ACAA1', 'ACACA', 'ACACB', 'ACADL', 'ACADM', 'ACADS', 'ACADSB', 'ACADVL', 'ACAT1', 'ACAT2', 'ASIC2', 'ASIC1', 'ACHE', 'DNAJC19', 'ACLY', 'ACO1', 'ACR', 'ACO2', 'ACOX1', 'ACP1', 'ACP2', 'ACP5', 'ACP3', 'ACRV1', 'DSTNP4', 'ACTA1', 'ACTA2', 'ACTB', 'ACTC1', 'ACTG1', 'ACTG2', 'OTOL1', 'ACTN4', 'ACTL6A', 'ACTN1', 'ACTN2', 'ACTN3', 'ACVR1', 'ACVR1B', 'ACVR2A', 'ACVR2B', 'ACVRL1', 'ACY1', 'ACYP1', 'ACYP2', 'ADA', 'ADAM8', 'ADAM10', 'ADAR', 'ADARB1', 'ADARB2', 'FAM3D', 'ADCY1', 'ADCY2', 'ADCY3', 'ADCY5', 'ADCY6', 'ADCY7', 'ADCY8', 'ADCY9', 'ADCYAP1', 'ADCYAP1R1', 'ADD1', 'ADD2', 'ADD3', 'ABCA4', 'PLIN2', 'ADH1A', 'ADH1B', 'ADH1C', 'ADH4', 'ADH5', 'ADH6', 'ADH7', 'ADK', 'ADM', 'ADORA1', 'ADORA2A', 'ADORA2B', 'VSTM4', 'PAOX', 'ADORA3', 'ADPRH', 'PARP1', 'PARP4', 'ADRA1D', 'ADRA1B', 'ADRA1A', 'ADRA2A',

In [8]:
#load top30TFs (in ensemble ID)
_ensID_top30TFs = pd.read_csv('ens_ID_top30TFs.csv', sep='\t', header=None)
ensID_top30TFs = list(_ensID_top30TFs[0])

In [9]:
#get gene_info for all top 30 TFs
mg = mygene.MyGeneInfo()
tfs_gene_info = mg.getgenes(ensID_top30TFs, fields='symbol')

INFO:biothings.client:querying 1-31...
INFO:biothings.client:done.


In [10]:
#create a dictionary with ensemble ID to symbol conversation for all top 30 TFs
tfs_ensemble2symbol = dict()
for d in tfs_gene_info:
    try:
      tfs_ensemble2symbol[d['query']] = d['symbol']
    except:
      pass

symbols_top30TFs = list(tfs_ensemble2symbol.values())
print(symbols_top30TFs)

['TAAR1', 'PNMA6E', 'MTNR1A', 'OLIG3', 'SFTA2', 'NKX2-6', 'SAA2-SAA4', 'IGFL1', 'CCDC169-SOHLH2', 'FOXB2', 'OR2T10', 'IQCF6', 'LDHC', 'H3Y1', 'APOA2', 'DYTN', 'PRLH', 'TSHB', 'OR14I1', 'FGB', 'REG1A', 'LBP', 'SPRR3', 'OBP2A', 'MT1B', 'CRYBA4', 'FOXI1', 'GLYATL3']


In [11]:
#determine TF genes that can be found in the hippie network
intersec_genes = [x for x in symbols_top30TFs if x in hippie_symbols] #top30 coding >> #intersection=22; top 10 coding = top 100 >> #intersection=4
print(intersec_genes)

['TAAR1', 'MTNR1A', 'OLIG3', 'SFTA2', 'NKX2-6', 'IGFL1', 'OR2T10', 'LDHC', 'APOA2', 'DYTN', 'PRLH', 'TSHB', 'OR14I1', 'FGB', 'REG1A', 'LBP', 'SPRR3', 'OBP2A', 'MT1B', 'CRYBA4', 'FOXI1', 'GLYATL3']


In [12]:
#get entrez ID for intersection genes

def get_keys_from_value(d, val):
  return {entrez: symbol for symbol, entrez in d.items() if entrez in val}

intersec_symbol2entrez = get_keys_from_value(hippie_entrez2symbol, intersec_genes)

In [13]:
#define methylation genes (DMTs) and create a dictionary with symbol to entrez ID conversation
methylation_genes = ['DNMT1', 'DNMT3A', 'DNMT3B', 'TET1', 'TET2', 'TET3']
dmts_symbol2entrez = {
    'DNMT1': 1786,
    'DNMT3A': 1788,
    'DNMT3B': 1789,
    'TET1': 80312,
    'TET2': 54790,
    'TET3': 200424
}

In [14]:
#collect all neighbors for DMTs and TFs
dmts_neighbors = {entrez: list(hippie_nx.neighbors(entrez)) for symbol, entrez in dmts_symbol2entrez.items()}
tfs_neighbors = {entrez: list(hippie_nx.neighbors(entrez)) for symbol, entrez in intersec_symbol2entrez.items()}

In [15]:
#get degrees of DMTs
dmts_degrees_df = pd.DataFrame(data={
    'gene': [dmts_symbol2entrez[g] for g in methylation_genes],
    'degree': [len(dmts_neighbors[dmts_symbol2entrez[g]]) for g in methylation_genes]
})
display(dmts_degrees_df)

,gene,degree
0,1786,224
1,1788,97
2,1789,108
3,80312,17
4,54790,135
5,200424,50


In [16]:
#get degrees of TFs
tfs_degrees_df = pd.DataFrame(data={
    'gene': [intersec_symbol2entrez[g] for g in intersec_genes],
    'degree': [len(tfs_neighbors[intersec_symbol2entrez[g]]) for g in intersec_genes]
})
display(tfs_degrees_df)

,gene,degree
0,134864,5
1,4543,166
2,167826,31
3,389376,8
4,137814,1
5,374918,12
6,127069,3
7,3948,36
8,336,90
9,391475,1


In [17]:
dmts_degrees = {dmts_symbol2entrez[g]: len(dmts_neighbors[dmts_symbol2entrez[g]]) for g in methylation_genes}
tfs_degrees = {intersec_symbol2entrez[g]: len(tfs_neighbors[intersec_symbol2entrez[g]]) for g in intersec_genes}

In [18]:
#get degrees of all genes in the hippie PPI network
hippie_degrees = {node: val for (node, val) in hippie_nx.degree(hippie_nx.nodes)}

In [19]:
#create a dictionary that collects all genes in the hippie network, that have the same degrees as the TFs; form: {tf_gene: [nodes in hippie with same node degree]}
same_node_degree = {k_tf:[k_hippie for k_hippie, v_hippie in hippie_degrees.items() if v_hippie == v_tf] for k_tf, v_tf in tfs_degrees.items()}

In [20]:
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

In [21]:
#for tf_genes without nodes in the hippie network that have the same node degree, look for nodes with degree +- the node degeree of the tf
tfs_without_same_node_degree = get_keys_from_value(same_node_degree, [])
for tf in tfs_without_same_node_degree:
  degree = tfs_degrees[tf]
  more_than_zero = False
  i = 1
  while(more_than_zero == False):
      degreeplus1 = degree + i
      degreeminus1 = degree - i

      genes_plus = get_keys_from_value(hippie_degrees, degreeplus1)
      if len(genes_plus)>0:
        more_than_zero = True

      genes_minus = get_keys_from_value(hippie_degrees, degreeminus1)
      if len(genes_minus)>0:
        more_than_zero = True
      i+=1
  genes = genes_plus + genes_minus
  same_node_degree[tf] = genes

In [22]:
#tfs_degrees = {entrez_tf: degree}
#same_node_degree = {entrez_tf: [all nodes in hippie with same node degree]}

In [23]:
n_intersec_genes = len(intersec_symbol2entrez.values()) #number of intersection genes

In [28]:
p_v = []
for j in range(0,30):
  n = 1000
  n_sets_of_random_k_genes = [] #random set of k nodes from hippie_nx.nodes with same node degree as tfs_genes
  for i in range(1,n+1): #100 random sets
    random_genes = [random.choice(tuple(same_node_degree[k_tf])) for k_tf, v_tf in tfs_degrees.items()]
    n_sets_of_random_k_genes.append(random_genes)

  #calculate pagerank for tf_genes und all n sets of random_k_genes
  person = {v: 1/len(dmts_symbol2entrez.values()) for v in dmts_symbol2entrez.values()}
  pr = nx.pagerank(hippie_nx, personalization = person)

  summe_tf = 0
  for k in intersec_symbol2entrez.values():
    summe_tf += pr[k]
  mean_tf = summe_tf/n_intersec_genes

  summen_random = []
  mean_random = []
  for i in range(0,n):
    summe = 0
    for k in n_sets_of_random_k_genes[i]:
      summe += pr[k]
    summen_random.append(summe)
    mean_random.append(summe/n_intersec_genes)

  geq = 0
  for i in range(0,n):
    if(mean_random[i] >= mean_tf):
      geq += 1
  p_value = (geq+1)/(n+1)
  p_v.append(p_value)

{216: 1.5235383992019084e-05, 3679: 4.013863557852843e-05, 1134: 1.3285913984584769e-05, 55607: 4.0311702145926234e-05, 71: 0.0001474013259283433, 5552: 1.084402634120414e-05, 960: 8.427072507069855e-05, 2886: 5.406298851365249e-05, 2064: 0.00022662785621003703, 5058: 0.00015776040313972284, 1742: 6.483555948113755e-05, 5296: 0.00010119325657353471, 26469: 2.5888432912159557e-05, 55914: 7.599447076885445e-05, 64750: 8.755434261730835e-05, 394: 1.6480604045653138e-05, 4771: 0.0001804676355334571, 3732: 1.733805564622731e-05, 54206: 4.602835041566164e-05, 4316: 1.4406556877753975e-05, 10140: 2.6124417250990407e-05, 4585: 5.040438730059861e-06, 9463: 0.00030252009680762124, 10628: 9.077337464038948e-05, 11218: 0.00010202629258571636, 2117: 5.6171373613678144e-05, 7088: 0.00013095876225937135, 2290: 4.676066427196849e-05, 7090: 9.921718571110046e-05, 3065: 0.0010539670900680225, 4086: 0.0002771532173198382, 10765: 6.302743839382778e-05, 920: 5.471914194167672e-05, 1601: 5.3588010801960166e

In [27]:
print(min(p_v))
print(max(p_v))
print(np.mean(p_v))

#Top 30 coding:
#min = 0.054945054945054944
#max = 0.08891108891108891
#mean = 0.07227772227772229

0.054945054945054944
0.08891108891108891
0.07227772227772229
